In [3]:
import os
from tqdm import tqdm 
from KGBuilder.config import *
from KGBuilder.data_utils import *

# args["root_dir"] = os.getcwd()
input_filepath = os.path.join(
    args["root_dir"], 
    args["model_dir"], "output",
    "event_filter-output-20220614.json"
)
# output_filepath = os.path.join(
#     args["root_dir"], 
#     args["model_dir"], "output",
#     "event_filter-output-20220614.json"
# )
data = common_data_loader(filepath=input_filepath)


Success: User custom arguments loaded.


In [4]:
#%%
from ltp import LTP
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from opencc import OpenCC
tw2sp = OpenCC('tw2sp')
s2twp = OpenCC('s2twp')
ltp_model = LTP("base2")


In [153]:
zhcn_sent = tw2sp.convert(data[96]["article"])
zhcn_sent

'陈时中确诊有重症危险？ 黄立民：第5天是关键 | 新冠肺炎防疫 | 要闻 | 经济日报中央流行疫情指挥中心指挥官陈时中今日通过指挥中心宣布确诊新冠肺炎，目前正在居家隔离中。不过，陈时中今年已年届68岁，年龄上属于高风险族群，染疫后的健康状况备受关注。台大儿童医院院长黄立民就表示，陈时中最主要会列入高风险族群在于「抽烟」因素，即使最近有戒烟行为，但时间仍短，隔离期间仍要注意身体，尤其是在确诊第5天，观察是否出现重症前驱症状。陈时中年龄已届68岁，先前又曾在立法院备询时，自曝过去曾装心脏支架，属于可用使用抗病毒药物Paxlovid高风险族群。不过，3月时他也宣布加入戒烟的行列中，烟龄50多年的陈时中虽说最终戒烟失败，但他仍「自认成功」。 黄立民表示，部长会是高风险的因素，主要还是「抽烟」，即使有戒烟但时间不长，肺部的功能还没恢复，仍有相当的重症风险，最少要戒烟1、2年，才比较能够降低风险。黄立民也指出，除了抽烟，部长的年龄以及心血管疾病也都是可能引发重症的风险，要立即服用Paxlovid口服药外，还要注意身体的状况，观察有没有重症前驱症状。「发病第五天开始要小心。」黄立民说，高风险族群染疫后要随时观察病情变化，像是会不会容易喘、精神状态不佳、血氧浓度下降、痰增多等等，若有出现相关症状，就表示病患的急性期相对严重；若是发烧的话，也要看有没有烧超过3天，一但超过就要立即进行进一步的治疗。※ 提醒您：吸烟有碍健康陈时中今传出确诊。图／指挥中心提供'

### Trace TripletExtraction module

In [154]:
def split_sents(content: str):
    """ 
    文章分句處理，切分長句、冒號、分號、感嘆號等做切分標識 
    - content: 輸入文本
    """
    return [sentence for sentence in re.split(r'[？?！!。；;：:\n\r]', content) if sentence]


def format_labelrole(ltp_model, hidden):
    """
    語義角色標註
    - hidden: 隱藏層權重字典
    dict_keys(['word_cls', 'word_input', 'word_length', 'word_cls_input', 'word_cls_mask'])
    """
    roles = ltp_model.srl(hidden, keep_empty=False)[0]
    roles_dict = {}
    for role in roles:
        roles_dict[role[0]] = {arg[0]: [arg[0], arg[1], arg[2]] for arg in role[1]}
    return roles_dict


def build_parse_child_dict(words, postags, arcs):
    """
    句法分析：為句子中的每個詞語，維護一個保存句法依存子節點的字典
    - words: LTP 分詞結果
    - postags: LTP 詞性結果
    - arcs: LTP 依存句法分析的結果
    """
    child_dict_list = []
    format_parse_list = []
    for index in range(len(words)):
        child_dict = dict()
        for arc_index in range(len(arcs)):
            # arcs 的索引從 1 開始 (head, tail, relation)
            if arcs[arc_index][0] == index+1:
                if arcs[arc_index][2] in child_dict:
                    child_dict[arcs[arc_index][2]].append(arc_index)
                else:
                    child_dict[arcs[arc_index][2]] = []
                    child_dict[arcs[arc_index][2]].append(arc_index)
        child_dict_list.append(child_dict)
    # 提取依存父節點 id
    rely_id = [arc[0] for arc in arcs]
    # 提取依存關係
    relation = [arc[2] for arc in arcs]
    # 匹配依存父節點詞語
    heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]
    for i in range(len(words)):
        # ['ATT', '李克強', 0, 'nh', '總理', 1, 'n']
        a = [relation[i], words[i], i, postags[i], heads[i], rely_id[i]-1, postags[rely_id[i]-1]]
        format_parse_list.append(a)
    return child_dict_list, format_parse_list


def ruler1(words, postags, roles_dict, role_index):
    """ 
    根據語意角色標註，直接獲取主詞、觸發詞、受詞之事件三元組 (基于語意角色 A0,A1,A2)
    - words: 分詞結果
    - postags: 詞性結果
    - roles_dict: 語義角色標注結果
    - role_index: 分詞索引號
    """
    v = words[role_index]  # 語意角色的觸發詞（動詞）
    role_info = roles_dict[role_index]
    # 保留同時有 A0(主詞) 或 A1(受詞) 及 ARGM-TMP(時間) 或 ARGM-LOC(地點) 的組合
    if any(x in role_info.keys() for x in ['A0', 'A1']):# and any(x in role_info.keys() for x in ['ARGM-TMP', 'ARGM-LOC']):
        _svotl = []
        for type in ['A0', 'VERB', 'A1', 'ARGM-TMP', 'ARGM-LOC']:
            if type != 'VERB':
                _words = ''.join(
                    [words[word_index] for word_index in range(role_info[type][1], role_info[type][2]+1) if words[word_index]]
                ) if type in role_info.keys() else ''
            else:
                _words = v
            _svotl.append(_words)
        if _svotl:
            return '1', _svotl
    return '4', []


def ruler2(words, postags, child_dict_list, arcs, roles_dict):
    """ 
    三元組抽取補充函式：依存句法分析 
    - words: LTP 分詞結果
    - postags: LTP 詞性結果
    - child_dict_list: 依存句法分析的子節點集合
    - arcs: LTP 依存句法分析的結果
    - roles_dict: 語義角色標注結果
    """
    svos = []
    for index in range(len(postags)):
        tmp = 1
        # 使用 rule1 的主謂賓三元组抽取結果
        if index in roles_dict:
            flag, triple = ruler1(words, postags, roles_dict, index)
            if flag == '1':
                svos.append(triple)
                tmp = 0
        if tmp == 1:
            # 若無 rule1 語意角色標記，則使用依存句法進行抽取
            if postags[index]:
                # 抽取以事件觸發詞為中心的 事件三元組
                child_dict = child_dict_list[index]
                # 主謂賓
                if 'SBV' in child_dict and 'VOB' in child_dict:
                    r = words[index]
                    e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
                    e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                    svos.append([e1, r, e2])
                # 定語後置，動賓關係
                relation = arcs[index][0]
                head = arcs[index][2]
                if relation == 'ATT':
                    if 'VOB' in child_dict:
                        e1 = complete_e(words, postags, child_dict_list, head - 1)
                        r = words[index]
                        e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                        temp_string = r + e2
                        if temp_string == e1[:len(temp_string)]:
                            e1 = e1[len(temp_string):]
                        if temp_string not in e1:
                            svos.append([e1, r, e2])
                # 含有介賓關係的主謂動補關係
                if 'SBV' in child_dict and 'CMP' in child_dict:
                    e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
                    cmp_index = child_dict['CMP'][0]
                    r = words[index] + words[cmp_index]
                    if 'POB' in child_dict_list[cmp_index]:
                        e2 = complete_e(words, postags, child_dict_list, child_dict_list[cmp_index]['POB'][0])
                        svos.append([e1, r, e2])
    return svos

def complete_e(words, postags, child_dict_list, word_index):
    """ 
    對找出的主語或者賓語進行擴展
    - words: LTP 分詞結果
    - postags: LTP 詞性結果
    - child_dict_list: 依存句法分析的子節點集合
    - word_index: 詞索引
    """
    child_dict = child_dict_list[word_index]
    prefix = ''
    if 'ATT' in child_dict:
        for i in range(len(child_dict['ATT'])):
            prefix += complete_e(words, postags, child_dict_list, child_dict['ATT'][i])
    postfix = ''
    if postags[word_index] == 'v':
        if 'VOB' in child_dict:
            postfix += complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
        if 'SBV' in child_dict:
            prefix = complete_e(words, postags, child_dict_list, child_dict['SBV'][0]) + prefix
    return prefix + words[word_index] + postfix


def parser_main(ltp_model, sentence):
    """
    事件三元組標注的主函数
    - sentence: 要進行事件三元組標注的文本陣列
    """
    # 分詞
    words, hidden = list(ltp_model.seg(sentence))
    # 詞性標註
    postags = list(ltp_model.pos(hidden))[0]
    # 依存句法
    arcs = ltp_model.dep(hidden)[0]
    child_dict_list, format_parse_list = build_parse_child_dict(words[0], postags, arcs)
    roles_dict = format_labelrole(hidden)
    return words[0], postags, child_dict_list, roles_dict, format_parse_list


def triples_main(content: str):
    """
    事件三元組標註主程式
    - content: 要進行事件三元組標注的文本
    """
    sentences = split_sents(content)
    svos = []
    for sentence in sentences:
        words, postags, child_dict_list, roles_dict, arcs = parser.parser_main([sentence])
        svo = ruler2(words, postags, child_dict_list, arcs, roles_dict)
        svos += svo
    return svos



In [155]:
# sentences = split_sents(zhcn_sent)
sentence = [zhcn_sent]
svos = []
# 分詞
words, hidden = list(ltp_model.seg(sentence))
words = words[0]
# 詞性標註
postags = list(ltp_model.pos(hidden))[0]
# 依存句法
arcs = ltp_model.dep(hidden)[0]
# 
child_dict_list, format_parse_list = build_parse_child_dict(words, postags, arcs)
acrs = format_parse_list.copy()
roles_dict = format_labelrole(ltp_model, hidden)

# svos = ruler2(words, postags, child_dict_list, arcs, roles_dict)
# for index in roles_dict:
#     svo = ruler1(words, postags, roles_dict, role_index=index)
#     svos.append(svo)
for index in range(len(postags)):
    if postags[index]:
        # 抽取以事件觸發詞為中心的 事件三元組
        child_dict = child_dict_list[index]
        # 主謂賓
        if 'SBV' in child_dict and 'VOB' in child_dict:
            r = words[index]
            e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
            e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
            svos.append([e1, r, e2])
        # 定語後置，動賓關係
        relation = arcs[index][0]
        head = arcs[index][2]
        if relation == 'ATT':
            if 'VOB' in child_dict:
                e1 = complete_e(words, postags, child_dict_list, head - 1)
                r = words[index]
                e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                temp_string = r + e2
                if temp_string == e1[:len(temp_string)]:
                    e1 = e1[len(temp_string):]
                if temp_string not in e1:
                    svos.append([e1, r, e2])
        # 含有介賓關係的主謂動補關係
        if 'SBV' in child_dict and 'CMP' in child_dict:
            e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
            cmp_index = child_dict['CMP'][0]
            r = words[index] + words[cmp_index]
            if 'POB' in child_dict_list[cmp_index]:
                e2 = complete_e(words, postags, child_dict_list, child_dict_list[cmp_index]['POB'][0])
                svos.append([e1, r, e2])

In [156]:
svos

[]

In [157]:
child_dict_list

[{'SBV': [0]},
 {'ADV': [1]},
 {'VOB': [2]},
 {'ATT': [3]},
 {'VOB': [4]},
 {'WP': [5]},
 {'SBV': [6]},
 {'WP': [7]},
 {'ATT': [8]},
 {'SBV': [9]},
 {'VOB': [10]},
 {'VOB': [11]},
 {'WP': [12]},
 {'ATT': [13]},
 {'ATT': [14]},
 {'ADV': [15]},
 {'WP': [16]},
 {'COO': [17]},
 {'WP': [18]},
 {'ATT': [19]},
 {'ATT': [20]},
 {'ATT': [21]},
 {'ATT': [22]},
 {'ATT': [23]},
 {'ATT': [24]},
 {'ATT': [25]},
 {'ATT': [26]},
 {'SBV': [27]},
 {'ADV': [28]},
 {'ADV': [29]},
 {'ATT': [30]},
 {'POB': [31]},
 {'VOB': [32]},
 {'VOB': [33]},
 {'VOB': [34]},
 {'WP': [35]},
 {'ADV': [36]},
 {'ADV': [37]},
 {'COO': [38]},
 {'ADV': [39]},
 {'ATT': [40]},
 {'VOB': [41]},
 {'WP': [42]},
 {'ADV': [43]},
 {'WP': [44]},
 {'SBV': [45]},
 {'ADV': [46]},
 {'ADV': [47]},
 {'SBV': [48]},
 {'COO': [49]},
 {'ATT': [50]},
 {'VOB': [51]},
 {'WP': [52]},
 {'ATT': [53]},
 {'ADV': [54]},
 {'COO': [55]},
 {'ATT': [56]},
 {'ATT': [57]},
 {'VOB': [58]},
 {'WP': [59]},
 {'ATT': [60]},
 {'ATT': [61]},
 {'RAD': [62]},
 {'ATT': [63

### 熟悉 LTP models

In [159]:
zhcn_sent

'陈时中确诊有重症危险？ 黄立民：第5天是关键 | 新冠肺炎防疫 | 要闻 | 经济日报中央流行疫情指挥中心指挥官陈时中今日通过指挥中心宣布确诊新冠肺炎，目前正在居家隔离中。不过，陈时中今年已年届68岁，年龄上属于高风险族群，染疫后的健康状况备受关注。台大儿童医院院长黄立民就表示，陈时中最主要会列入高风险族群在于「抽烟」因素，即使最近有戒烟行为，但时间仍短，隔离期间仍要注意身体，尤其是在确诊第5天，观察是否出现重症前驱症状。陈时中年龄已届68岁，先前又曾在立法院备询时，自曝过去曾装心脏支架，属于可用使用抗病毒药物Paxlovid高风险族群。不过，3月时他也宣布加入戒烟的行列中，烟龄50多年的陈时中虽说最终戒烟失败，但他仍「自认成功」。 黄立民表示，部长会是高风险的因素，主要还是「抽烟」，即使有戒烟但时间不长，肺部的功能还没恢复，仍有相当的重症风险，最少要戒烟1、2年，才比较能够降低风险。黄立民也指出，除了抽烟，部长的年龄以及心血管疾病也都是可能引发重症的风险，要立即服用Paxlovid口服药外，还要注意身体的状况，观察有没有重症前驱症状。「发病第五天开始要小心。」黄立民说，高风险族群染疫后要随时观察病情变化，像是会不会容易喘、精神状态不佳、血氧浓度下降、痰增多等等，若有出现相关症状，就表示病患的急性期相对严重；若是发烧的话，也要看有没有烧超过3天，一但超过就要立即进行进一步的治疗。※ 提醒您：吸烟有碍健康陈时中今传出确诊。图／指挥中心提供'

In [163]:
#%%
ws, hidden = ltp_model.seg([zhcn_sent])
ws_list = ws[0].copy()
srl_list = ltp_model.srl(hidden, keep_empty=False)[0]
dep_list = ltp_model.dep(hidden)[0]


In [164]:
srl_list


[(1, [('A1', 0, 0)]),
 (2, [('A0', 0, 0), ('A1', 3, 4)]),
 (10, [('A0', 8, 9), ('A1', 11, 11), ('A1', 85, 88)]),
 (32,
  [('A0', 19, 27),
   ('ARGM-TMP', 28, 28),
   ('ARGM-MNR', 29, 31),
   ('A1', 33, 34),
   ('ARGM-ADV', 90, 94),
   ('ARGM-ADV', 272, 272)]),
 (38,
  [('ARGM-TMP', 36, 36),
   ('ARGM-ADV', 37, 37),
   ('A1', 39, 41),
   ('ARGM-DIS', 96, 96),
   ('A0', 97, 97),
   ('ARGM-TMP', 287, 290)]),
 (48, [('A1', 50, 51)]),
 (49,
  [('ARGM-DIS', 43, 43),
   ('A0-CRD', 45, 48),
   ('A1', 50, 51),
   ('A1', 126, 127),
   ('A0', 298, 300)]),
 (55,
  [('A0', 45, 45), ('A0', 53, 54), ('A1', 56, 58), ('ARGM-TMP', 101, 102)]),
 (65,
  [('A0', 60, 64), ('A1', 66, 66), ('ARGM-TMP', 101, 102), ('A1', 116, 120)]),
 (74,
  [('A0', 68, 72),
   ('ARGM-DIS', 73, 73),
   ('A1', 76, 187),
   ('A0', 189, 189),
   ('A1', 192, 241),
   ('A0', 243, 243),
   ('ARGM-ADV', 244, 244),
   ('A1', 247, 292),
   ('A0', 295, 295),
   ('A1', 298, 320)]),
 (80, [('A0', 76, 76), ('ARGM-ADV', 77, 78), ('A2', 81, 

In [166]:
# tag_list = []
# for idx, val in enumerate(srl_list):
srl_ws_list = []
for _set in srl_list:
    head = [("head", _set[0], _set[0])]
    _set = head + _set[1]
    srl_ws_list.append([(_srl[0], ws_list[_srl[1]:_srl[2]+1]) for _srl in _set])


#### 印出每個分詞結果的語意角色

In [167]:
for _list in srl_ws_list:
    text = " ".join(["".join(_set[1]) for _set in _list])
    tag = [_set[0] for _set in _list]
    print(text)
    print(tag)
    print("\n")

确诊 陈时中
['head', 'A1']


有 陈时中 重症危险
['head', 'A0', 'A1']


是 第5天 关键 「抽烟」因素
['head', 'A0', 'A1', 'A1']


宣布 经济日报中央流行疫情指挥中心指挥官陈时中 今日 通过指挥中心 确诊新冠肺炎 即使最近有戒烟行为 还要
['head', 'A0', 'ARGM-TMP', 'ARGM-MNR', 'A1', 'ARGM-ADV', 'ARGM-ADV']


在 目前 正 居家隔离中 但 时间 发病第五天开始
['head', 'ARGM-TMP', 'ARGM-ADV', 'A1', 'ARGM-DIS', 'A0', 'ARGM-TMP']


年 68岁
['head', 'A1']


届 不过 陈时中今年已年 68岁 68岁 高风险族群
['head', 'ARGM-DIS', 'A0-CRD', 'A1', 'A1', 'A0']


属于 陈时中 年龄上 高风险族群 隔离期间
['head', 'A0', 'A0', 'A1', 'ARGM-TMP']


备受 染疫后的健康状况 关注 隔离期间 是否出现重症前驱症状
['head', 'A0', 'A1', 'ARGM-TMP', 'A1']


表示 台大儿童医院院长黄立民 就 陈时中最主要会列入高风险族群在于「抽烟」因素，即使最近有戒烟行为，但时间仍短，隔离期间仍要注意身体，尤其是在确诊第5天，观察是否出现重症前驱症状。陈时中年龄已届68岁，先前又曾在立法院备询时，自曝过去曾装心脏支架，属于可用使用抗病毒药物Paxlovid高风险族群。不过，3月时他也宣布加入戒烟的行列中，烟龄50多年的陈时中虽说最终戒烟失败，但他仍「自认成功」 黄立民 部长会是高风险的因素，主要还是「抽烟」，即使有戒烟但时间不长，肺部的功能还没恢复，仍有相当的重症风险，最少要戒烟1、2年，才比较能够降低风险 黄立民 也 除了抽烟，部长的年龄以及心血管疾病也都是可能引发重症的风险，要立即服用Paxlovid口服药外，还要注意身体的状况，观察有没有重症前驱症状。「发病第五天开始要小心 黄立民 高风险族群染疫后要随时观察病情变化，像是会不会容易喘、精神状态不佳
['head', 'A0', 'ARGM-DIS', 'A1', 'A0', 'A

In [ ]:
# %%
srl_ws_list = []
for _srl in tqdm(srl_list):
    _new_srl = []
    if _srl:
        for _srl_set in _srl:
            item = (_srl_set[0], ws_list[_srl_set[1]], ws_list[_srl_set[2]])
            _new_srl.append(item)
    srl_ws_list.append(_new_srl)
srl_ws_list

# %%
dep_ws_list = []
for _dep in tqdm(dep_list):
    item = (ws_list[_dep[0]], ws_list[_dep[1]], _dep[2])
    dep_ws_list.append(item)
dep_ws_list



### 測試 event_filter 篩選

In [6]:
data[100]["article"]

'雷昇主攻音樂劇作曲 接受不同意見修改再修改 | 政經焦點 | 要聞 | 經濟日報音樂劇作曲家雷昇是美國紐約大學帝許藝術學院創作碩士，他也是台灣第一位赴該校取得音樂劇創作學位的作曲人，他說，不要害怕觀眾反饋給負面意見，修改再修改是百老匯常態。雷昇說，他在紐約學到一點就是「修改很重要」，「音樂劇是跟觀眾之間的對話，不是自己說了算，如何在調整的前提之下，又不能失去本來想要表達的，這是最難的部分。」 34歲的雷昇接受中央社記者專訪時表示，小時候學鋼琴，高中時學吉他也開始玩創作，大學時只是因為分數到了，所以念了政大國貿，但知道自己不喜歡，大三原本想轉系，最後還是把書念完，開始尋找自我。雷昇說，自己本來對於文學比較有興趣，在學長的建議下念了台大戲劇研究所，認識了同班同學也是音樂劇「熱帶天使」編劇林孟寰，兩人開始了長達10年的合作情誼至今。父親是曾任明基電通全球策略長與營銷總部總經理的雷輝，從小至今，雷輝夫婦總會帶著全家到世界各地去聽歌劇和古典音樂，但雷昇聽歌劇或音樂會還是沒那麼投入，「我還是比較喜歡有人物、有情節的戲劇，再加上自己對音樂的熱情，希望能朝以音樂為主幹的創作邁進。」這些加總起來，目標清晰就是音樂劇，於是他赴美攻讀音樂劇作曲創作。「我們上課就是實作，考進去時分成兩組，文字組跟音樂組，每次都會隨機分組創作，拿到觀眾回饋意見，再進行修改。」雷昇說，持續不停反覆的訓練過程，「創作就會有一種直覺，拿到歌詞就大概知道音樂可以怎麼走，一定都寫得出來，只是好不好聽。」在美國紐約大學拿到表演碩士的音樂劇導演曾慧誠表示，帝許藝術學院（Tisch School of the Arts）對台灣人來說比較陌生，但它在美國表演藝術領域上與耶魯大學、茱莉亞音樂院並列，是很不容易進入的學院，「雷昇是台灣至今唯一一位台灣人真正進入音樂劇創作學院訓練出來的作曲家，他的表現可以期待。」'

In [7]:
[" ".join(_set["event"]) for _set in data[100]["event_triplets"] if not _set["filter_flag"]]


['雷昇 主攻 音樂劇作曲',
 '雷昇 接受 不同意見',
 '他 是 台灣第一位赴該校取得音樂劇創作學位作曲人',
 '作曲人 赴 該校',
 '作曲人 取得 音樂劇創作學位',
 '他 說 不要害怕觀眾反饋給負面意見修改再修改是百老匯常態',
 '觀眾 反饋 負面意見',
 '修改再修改 是 百老匯常態',
 '他 學 一點',
 '他在紐約學到一點 是 修改很重要',
 '音樂劇 是 跟觀眾之間對話',
 '這 是 最難部分',
 '自己 有 興趣',
 '自己 念 台大戲劇研究所',
 '自己 認識 同班同學也是音樂劇熱帶天使編劇林孟寰',
 '合作情誼 達 10年',
 '父親 是 曾任明基電通全球策略長與營銷總部總經理雷輝',
 '雷輝夫婦 帶 全家',
 '雷輝夫婦 到 世界各地',
 '雷輝夫婦 聽 歌劇和古典音樂',
 '雷昇 投入 聽歌劇或音樂會',
 '戲劇 有 人物',
 '戲劇 有 情節',
 '我 加上 自己對音樂熱情',
 '我 希望 能朝以音樂為主幹創作邁進',
 '創作 為 主幹',
 '目標清晰 是 音樂劇',
 '他 赴 美',
 '他 攻讀 音樂劇作曲創作',
 '創作 有 一種直覺',
 '曾慧誠 拿 表演碩士',
 '它 是 很不容易進入學院',
 '作曲家 進入 音樂劇創作學院',
 '台灣人 訓練 作曲家',
 '分數到了 因為 大學時只是',
 '分數到了， 因為-所以 念了政大國貿，但知道自己不喜歡，大三原本想轉系，最後還是把書念完，開始尋找自我。']